In [6]:
#Original code for this cell at https://github.com/theofil/dscout/blob/master/analysis/exampleAnalysis.ipynb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import subprocess
import bz2


plt.rc('font', size=15)



# data files
path = './data/hiion/'
files = os.listdir(path)
files = [f for f in files if 'monitor' not in f]

# getrunNumber from string e.g., int('/data/hiion/scout_326676_000000.monitor.txt'.split('_')[1]
getRun = lambda x: int(x.split('_')[1])



# list to hold dataframes
dfs = []
runIndex= []

for file in files:
    filepath = path+file
    df = pd.read_csv(filepath)
    
  
    # add a column with the run number
    df['run'] = getRun(file) 
    dfs += [df]
    
    #make in index with the run numbers
    runIndex += [getRun(file)]
plt.rc('figure', max_open_warning = 100)




In [7]:
#Fix the runs where the orbit number resets (the warning does not matter)

for df in dfs:
    if df['orbit'].tolist().index(df['orbit'].max())< df['orbit'].tolist().index(df['orbit'].min()):
        for i in range(len(df)):
            if df['orbit'][i]>df['orbit'][i+1]:
                tmp = i
                break
        df['orbit'][tmp+1:] = df['orbit'][tmp] + df['orbit'][tmp+1:]
            

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [8]:
badruns = [0, 1, 4, 8, 9, 11, 28, 30, 33, 34, 35, 36, 39, 46, 47, 50,  57, 65, 68, 69, 70, 72, 77, 78]
goodruns = [i for i in range(len(dfs)) if i not in badruns]

In [9]:
dfg = [] #data frame with only goodruns

for i in range(len(dfs)):
    if i in goodruns: dfg.append(dfs[i])

In [12]:
merged = pd.concat(dfg, ignore_index=True, sort=False)

In [ ]:
# for each orbit, we dump in the CSV 110 BX's (= bxMax - bxMin) that 'sit' in the LHC abort gap (end of the train)
# that is, each orbit in the CSV corresponds to 110*24.95 ns of active recorded time, due to our selection made 
# (we dump in CSV only a part of each orbit during the LHC abort gap)
timePerOrbit = 110*24.95
print('timePerOrbit %2.1f ns '%timePerOrbit)

timeUnit = 0.2*1e9 # desired time unit in seconds
orbitsPerTimeUnit = timeUnit/timePerOrbit
print('orbitsPerTimeUnit = %2.3f'%orbitsPerTimeUnit)





indxMin  = 0
indxMax  = len(merged)-1
rectime   = (indxMax-indxMin)*timePerOrbit # in ns
print('orbitMin %d orbitMax %d'%(indxMin, indxMax))
print('nOrbits %d '%(indxMax - indxMin))
print('total recording time %2.4f ns (~= %2.1f s)' %(rectime, rectime*1.e-9))

bins=int((indxMax-indxMin)/orbitsPerTimeUnit) 
timePerBin = rectime/bins

muons=np.zeros(bins)
sclice=np.array([indxMin+ ((indxMax-indxMin)/bins)*i for i in range(bins+1)])
for orbit in merged['orbit']:
    for i in range(bins):
        if(orbit >= sclice[i] and orbit < sclice[i+1]): # note the >=, the = is needed to capture first orbit
            muons[i]+=1
        if(orbit == merged['orbit'][len(merged)-1] and orbit == sclice[i+1]): # note the ==, is needed to capture last orbit
                                                        # last bin is [leftedge,rightedge] and not [leftedge,rightedge)
                muons[i]+=1

timePerOrbit 2744.5 ns 
orbitsPerTimeUnit = 72873.019
orbitMin 0 orbitMax 2108986
nOrbits 2108986 
total recording time 5788112077.0000 ns (~= 5.8 s)


129369    242177006
129369    241768987
129369    242167898
129369    242354558
129369    223753705
Name: orbit, dtype: int64